필요한 라이브러리 가져오기

In [4]:
import requests
import pandas as pd
import bs4
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


관광객 집계 데이터

In [2]:
with open('c:/Kcoding/MiniProject/APIkey.txt') as file:
    APIkey = file.read()

startdate = ['20220101', '20220201', '20220301', '20220401', '20220501', '20220601', '20220701', '20220801', '20220901', '20221001', '20221101', '20221201']
enddate = ['20220131', '20220228', '20220331', '20220430', '20220531', '20220630', '20220731', '20220831', '20220930', '20221031', '20221130', '20221231']

row_list = [] # 행값
value_list = [] # 데이터값

for s, e in tqdm(zip(startdate,enddate)):
    response = requests.get(f'https://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?numOfRows=13000&MobileOS=ETC&MobileApp=TestAPP&serviceKey={APIkey}&startYmd={s}&endYmd={e}')
    time.sleep(2)
    contents = response.text
    xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
    rows = xml_obj.findAll('item')
    for i in range(0, len(rows)): # xml 안의 데이터 수집
        columns = rows[i].find_all()
        for j in range(0,len(columns)): # 첫째 행 데이터 수집
            value_list.append(columns[j].text) # 컬럼의 각 데이터 값 저장
        row_list.append(value_list) # 각 행의 value값 전체 저장
        value_list=[] # 데이터 리스트 값 초기화


12it [00:41,  3.46s/it]


In [3]:
# xml값 DataFrame으로 만들기
df = pd.DataFrame(row_list)

In [4]:
# 데이터 확인
df

,0,1,2,3,4,5,6,7
0,11,서울특별시,6,토요일,1,현지인(a),3698523.5,20220101
1,11,서울특별시,6,토요일,2,외지인(b),1171117.5,20220101
2,11,서울특별시,6,토요일,3,외국인(c),3806.8778575661786,20220101
3,26,부산광역시,6,토요일,1,현지인(a),1206936.5,20220101
4,26,부산광역시,6,토요일,2,외지인(b),445102.0,20220101
...,...,...,...,...,...,...,...,...
19817,48,경상남도,6,토요일,2,외지인(b),612486.0,20221231
19818,48,경상남도,6,토요일,3,외국인(c),556.3489851673804,20221231
19819,50,제주특별자치도,6,토요일,1,현지인(a),552450.5,20221231
19820,50,제주특별자치도,6,토요일,2,외지인(b),198383.5,20221231


In [5]:
# 중복값 제거
df.drop_duplicates(inplace=True, ignore_index=True)

In [6]:
# 컬럼명 변경
df.rename(columns={0:'areaCode', 1:'areaNm', 2:'daywkDivCd', 3:'daywkDivNm', 4:'touDivCd', 5:'touDivNm', 6:'touNum', 7:'baseYmd'}, inplace=True)
df

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
0,11,서울특별시,6,토요일,1,현지인(a),3698523.5,20220101
1,11,서울특별시,6,토요일,2,외지인(b),1171117.5,20220101
2,11,서울특별시,6,토요일,3,외국인(c),3806.8778575661786,20220101
3,26,부산광역시,6,토요일,1,현지인(a),1206936.5,20220101
4,26,부산광역시,6,토요일,2,외지인(b),445102.0,20220101
...,...,...,...,...,...,...,...,...
18236,48,경상남도,6,토요일,2,외지인(b),612486.0,20221231
18237,48,경상남도,6,토요일,3,외국인(c),556.3489851673804,20221231
18238,50,제주특별자치도,6,토요일,1,현지인(a),552450.5,20221231
18239,50,제주특별자치도,6,토요일,2,외지인(b),198383.5,20221231


In [7]:
# 관광객 유형별 종속변수로 만들기위하여 유형별로 나누어 각 리스트에 저장
local = []
outside = []
foreigner = []

for index, row in df.iterrows():
    if row['touDivNm'] == '현지인(a)':
        local.append(row['touNum'])   
    elif row['touDivNm'] == '외지인(b)':
        outside.append(row['touNum'])
    elif row['touDivNm'] == '외국인(c)':
        foreigner.append(row['touNum'])
    else:
        foreigner.append(row['touNum'])

In [8]:
# areaNm 컬럼을 기준으로 불필요한 유형 제거
df = df.loc[(df['touDivNm'] != '외지인(b)') & (df['touDivNm'] != '외국인(c)')]
df

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
0,11,서울특별시,6,토요일,1,현지인(a),3698523.5,20220101
3,26,부산광역시,6,토요일,1,현지인(a),1206936.5,20220101
6,27,대구광역시,6,토요일,1,현지인(a),1013701.5,20220101
9,28,인천광역시,6,토요일,1,현지인(a),1076911.5,20220101
12,29,광주광역시,6,토요일,1,현지인(a),616735.0,20220101
...,...,...,...,...,...,...,...,...
18226,45,전라북도,6,토요일,1,현지인(a),979002.5,20221231
18229,46,전라남도,6,토요일,1,현지인(a),930996.5,20221231
18232,47,경상북도,6,토요일,1,현지인(a),1341977.0,20221231
18235,48,경상남도,6,토요일,1,현지인(a),1599500.0,20221231


In [9]:
# 관광객 타입 변경
local = list(map(float, local))
outside = list(map(float, outside))
foreigner = list(map(float, foreigner))

In [10]:
# 외국인 관광객 결측치 평균으로 대체
foreigner_mean = sum(foreigner) / len(foreigner) # 외국인 평균값      
for i in range(17):
    foreigner.insert(3876, foreigner_mean) # 외국인 누락값 17개 평균값으로 추가

In [11]:
# 각 컬럼에 추가
df['local'] = local
df['outside'] = outside
df['foreigner'] = foreigner

In [12]:
# 인덱스값 재설정
df.set_index([pd.Index(range(0, 6086))], inplace=True)
df

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd,local,outside,foreigner
0,11,서울특별시,6,토요일,1,현지인(a),3698523.5,20220101,3698523.5,1171117.5,3806.877858
1,26,부산광역시,6,토요일,1,현지인(a),1206936.5,20220101,1206936.5,445102.0,2165.794258
2,27,대구광역시,6,토요일,1,현지인(a),1013701.5,20220101,1013701.5,264113.5,218.422286
3,28,인천광역시,6,토요일,1,현지인(a),1076911.5,20220101,1076911.5,496692.0,2528.373800
4,29,광주광역시,6,토요일,1,현지인(a),616735.0,20220101,616735.0,161028.0,100.034962
...,...,...,...,...,...,...,...,...,...,...,...
6081,45,전라북도,6,토요일,1,현지인(a),979002.5,20221231,979002.5,370013.0,287.105010
6082,46,전라남도,6,토요일,1,현지인(a),930996.5,20221231,930996.5,475352.5,340.188862
6083,47,경상북도,6,토요일,1,현지인(a),1341977.0,20221231,1341977.0,726484.5,277.971033
6084,48,경상남도,6,토요일,1,현지인(a),1599500.0,20221231,1599500.0,612486.0,556.348985


In [13]:
# 불필요한 컬럼삭제
df.drop(columns=['areaCode', 'daywkDivCd', 'touDivCd', 'touDivNm', 'touNum'], inplace=True)
df

,areaNm,daywkDivNm,baseYmd,local,outside,foreigner
0,서울특별시,토요일,20220101,3698523.5,1171117.5,3806.877858
1,부산광역시,토요일,20220101,1206936.5,445102.0,2165.794258
2,대구광역시,토요일,20220101,1013701.5,264113.5,218.422286
3,인천광역시,토요일,20220101,1076911.5,496692.0,2528.373800
4,광주광역시,토요일,20220101,616735.0,161028.0,100.034962
...,...,...,...,...,...,...
6081,전라북도,토요일,20221231,979002.5,370013.0,287.105010
6082,전라남도,토요일,20221231,930996.5,475352.5,340.188862
6083,경상북도,토요일,20221231,1341977.0,726484.5,277.971033
6084,경상남도,토요일,20221231,1599500.0,612486.0,556.348985


In [14]:
# 종합 관광객 컬럼 추가
df['total_tourist'] = df['local'] + df['outside'] + df['foreigner']

In [15]:
df['areaNm'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [16]:
df['daywkDivNm'].unique()

array(['토요일', '일요일', '월요일', '화요일', '수요일', '목요일', '금요일'], dtype=object)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6086 entries, 0 to 6085
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   areaNm         6086 non-null   object 
 1   daywkDivNm     6086 non-null   object 
 2   baseYmd        6086 non-null   object 
 3   local          6086 non-null   float64
 4   outside        6086 non-null   float64
 5   foreigner      6086 non-null   float64
 6   total_tourist  6086 non-null   float64
dtypes: float64(4), object(3)
memory usage: 333.0+ KB


In [154]:
# 년월일 데이터 타입 및 형식 통일화를 위한 변경
df['baseYmd'] = pd.to_datetime(df['baseYmd'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
df['baseYmd'] = pd.to_datetime(df['baseYmd'])

In [19]:
# csv 저장
df.to_csv('./data/tour_data.csv', index=False, encoding='utf-8-sig')
df

,areaNm,daywkDivNm,baseYmd,local,outside,foreigner,total_tourist
0,서울특별시,토요일,2022-01-01,3698523.5,1171117.5,3806.877858,4.873448e+06
1,부산광역시,토요일,2022-01-01,1206936.5,445102.0,2165.794258,1.654204e+06
2,대구광역시,토요일,2022-01-01,1013701.5,264113.5,218.422286,1.278033e+06
3,인천광역시,토요일,2022-01-01,1076911.5,496692.0,2528.373800,1.576132e+06
4,광주광역시,토요일,2022-01-01,616735.0,161028.0,100.034962,7.778630e+05
...,...,...,...,...,...,...,...
6081,전라북도,토요일,2022-12-31,979002.5,370013.0,287.105010,1.349303e+06
6082,전라남도,토요일,2022-12-31,930996.5,475352.5,340.188862,1.406689e+06
6083,경상북도,토요일,2022-12-31,1341977.0,726484.5,277.971033,2.068739e+06
6084,경상남도,토요일,2022-12-31,1599500.0,612486.0,556.348985,2.212542e+06


기상청 데이터

In [20]:
# 기상청 csv 형식 데이터 불러오기
weather = pd.read_csv('./data/OBS_ASOS_DD_20230317114852.csv', encoding='cp949')
weather.head()

,지점,지점명,일시,평균기온(°C),일강수량(mm),일 최심적설(cm)
0,101,춘천,2022-01-01,-8.2,NaN,NaN
1,101,춘천,2022-01-02,-2.7,0.0,NaN
2,101,춘천,2022-01-03,-3.8,0.0,NaN
3,101,춘천,2022-01-04,-3.4,NaN,NaN
4,101,춘천,2022-01-05,-4.7,NaN,NaN


In [21]:
# 정보 확인
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6203 entries, 0 to 6202
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   지점          6203 non-null   int64  
 1   지점명         6203 non-null   object 
 2   일시          6203 non-null   object 
 3   평균기온(°C)    6202 non-null   float64
 4   일강수량(mm)    2184 non-null   float64
 5   일 최심적설(cm)  140 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 290.9+ KB


In [22]:
# 지점명 변경을 위한 종류 확인
weather['지점명'].unique()

array(['춘천', '서울', '인천', '서산', '대전', '대구', '전주', '울산', '광주', '부산', '제주',
       '이천', '제천', '세종', '강진군', '봉화', '산청'], dtype=object)

In [23]:
# 각 지역에 맞게끔 변경
weather.replace('춘천', '강원도', inplace=True)
weather.replace('서울', '서울특별시', inplace=True)
weather.replace('인천', '인천광역시', inplace=True)
weather.replace('이천', '경기도', inplace=True)
weather.replace('대전', '대전광역시', inplace=True)
weather.replace('대구', '대구광역시', inplace=True)
weather.replace('전주', '전라북도', inplace=True)
weather.replace('울산', '울산광역시', inplace=True)
weather.replace('광주', '광주광역시', inplace=True)
weather.replace('부산', '부산광역시', inplace=True)
weather.replace('제주', '제주특별자치도', inplace=True)
weather.replace('제천', '충청북도', inplace=True)
weather.replace('서산', '충청남도', inplace=True)
weather.replace('세종', '세종특별자치시', inplace=True)
weather.replace('강진군', '전라남도', inplace=True)
weather.replace('산청', '경상남도', inplace=True)
weather.replace('봉화', '경상북도', inplace=True)

In [24]:
# 불필요한 컬럼 삭제
weather.drop(columns={'지점'}, inplace=True)

In [25]:
# 컬럼명 변경
weather.rename(columns={'지점명':'areaNm', '일시':'baseYmd', '평균기온(°C)':'avgTemp', '일강수량(mm)':'dayRf', '일 최심적설(cm)':'daySf'}, inplace=True)

In [26]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6203 entries, 0 to 6202
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   areaNm   6203 non-null   object 
 1   baseYmd  6203 non-null   object 
 2   avgTemp  6202 non-null   float64
 3   dayRf    2184 non-null   float64
 4   daySf    140 non-null    float64
dtypes: float64(3), object(2)
memory usage: 242.4+ KB


In [27]:
# 날짜 타입 변경
weather['baseYmd'] = pd.to_datetime(weather['baseYmd'])

In [28]:
# 결측값 0.0 으로 통일
weather.fillna(0.0, inplace=True)

In [29]:
# 확인
weather

,areaNm,baseYmd,avgTemp,dayRf,daySf
0,강원도,2022-01-01,-8.2,0.0,0.0
1,강원도,2022-01-02,-2.7,0.0,0.0
2,강원도,2022-01-03,-3.8,0.0,0.0
3,강원도,2022-01-04,-3.4,0.0,0.0
4,강원도,2022-01-05,-4.7,0.0,0.0
...,...,...,...,...,...
6198,경상남도,2022-12-27,-1.9,0.0,0.0
6199,경상남도,2022-12-28,1.0,0.0,0.0
6200,경상남도,2022-12-29,0.0,0.0,0.0
6201,경상남도,2022-12-30,0.5,0.0,0.0


In [30]:
# csv 저장
weather.to_csv('./data/weather.csv', encoding='utf-8')

지역축제 데이터

In [31]:
# csv 데이터 불러오기
festival = pd.read_csv("./data/전국문화축제표준데이터.csv", encoding='cp949')
festival.head(3)

,축제명,개최장소,축제시작일자,축제종료일자,축제내용,주관기관,주최기관,후원기관,전화번호,홈페이지주소,관련정보,소재지도로명주소,소재지지번주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,강원대학교 산학협력단 K-Digital Training 제2기 교육생 모집,강원대학교 춘천캠퍼스,2022-10-04,2023-03-13,언리얼엔진 기반 메타버스 개발자 양성 과정 교육생 모집,고용노동부,강원대학교 산학협력단,NaN,033-250-8945,https://www.cccf.or.kr/Home/H40000/H40400/boar...,행사+축제,강원도 춘천시 강원대학길 1,강원도 춘천시 효자동 192-1,37.868024,127.748387,2023-01-12,4180000,강원도 춘천시
1,자연과 영혼을 담은 구자근展,춘천미술관 전관,2022-09-02,2022-09-07,자연과 영혼을 담은,구자근,구자근,NaN,033-241-1856,https://www.cccf.or.kr/Home/H40000/H40400/boar...,전시관람,강원도 춘천시 서부대성로 71,강원도 춘천시 옥천동 73-2,37.882374,127.731335,2023-01-12,4180000,강원도 춘천시
2,한동국 현관문,개나리미술관,2022-09-02,2022-09-18,"개나리미술관에서 원주출신,한동국의 첫 개인전을 개최",한동국,강원문화재단,NaN,070-8095-3899,https://www.cccf.or.kr/Home/H40000/H40400/boar...,전시관람,강원도 춘천시 동내면 거두택지길44번길 7-19,강원도 춘천시 동내면 거두리 1123-6,37.854547,127.758295,2023-01-12,4180000,강원도 춘천시


In [32]:
# 가독성을 위해 불필요한 컬럼 삭제
festival.drop(columns=['개최장소', '축제내용', '주관기관', '주최기관', '후원기관', '전화번호', '홈페이지주소', '소재지지번주소', '위도', '경도', '데이터기준일자', '제공기관코드', '제공기관명'], inplace=True)
festival

,축제명,축제시작일자,축제종료일자,관련정보,소재지도로명주소
0,강원대학교 산학협력단 K-Digital Training 제2기 교육생 모집,2022-10-04,2023-03-13,행사+축제,강원도 춘천시 강원대학길 1
1,자연과 영혼을 담은 구자근展,2022-09-02,2022-09-07,전시관람,강원도 춘천시 서부대성로 71
2,한동국 현관문,2022-09-02,2022-09-18,전시관람,강원도 춘천시 동내면 거두택지길44번길 7-19
3,2022 춘천학 학술대회 댐과 춘천의 미래,2022-08-26,2022-08-26,행사+축제,강원도 춘천시 강원대학길 1
4,할말하않 황효창展,2022-08-16,2022-08-28,전시관람,강원도 춘천시 동내면 거두택지길44번길 7-19
...,...,...,...,...,...
1360,여주오곡나루축제,2022-10-21,2022-10-23,신륵사+명성황후생가+영릉+황학산수목원,경기도 여주시 신륵사길 7
1361,김해분청도자기축제,2022-10-21,2022-10-30,김해분청도자박물관+클레이아크미술관,경상남도 김해시 진례면 진례로 275-35
1362,청송사과축제,2022-11-02,2022-11-06,NaN,경상북도 청송군 청송읍 금월로 269
1363,영덕대게축제,2022-12-27,2022-12-27,해파랑공원,경상북도 영덕군 강구면 영덕대게로 136


In [33]:
# 축제만 추출
festival = festival[festival['축제명'].str.contains('축제', na =False)]

In [34]:
# 개최취소된 축제 삭제
festival = festival.loc[festival['관련정보'] != '개최취소']

In [35]:
# 개최일 타입 변경
festival['축제시작일자'] = pd.to_datetime(festival['축제시작일자'])
festival['축제종료일자'] = pd.to_datetime(festival['축제종료일자'])

In [36]:
# 2022년 축제만 추출
festival = festival[festival['축제시작일자'].dt.year == 2022]
festival = festival[festival['축제종료일자'].dt.year == 2022]
festival

,축제명,축제시작일자,축제종료일자,관련정보,소재지도로명주소
18,불갑산상사화축제,2022-09-16,2022-09-25,NaN,전라남도 영광군 불갑면 불갑사로 450
21,청송사과축제,2022-11-09,2022-11-13,NaN,NaN
23,필봉마을굿축제,2022-08-12,2022-08-15,NaN,전라북도 임실군 강진면 강운로 272
24,임실N치즈축제,2022-10-07,2022-10-10,NaN,전라북도 임실군 성수면 도인2길 50
25,산타축제,2022-12-23,2022-12-25,NaN,전라북도 임실군 성수면 도인2길 50
...,...,...,...,...,...
1358,광주세계김치축제,2022-10-20,2022-10-23,NaN,광주광역시 남구 김치로 60
1360,여주오곡나루축제,2022-10-21,2022-10-23,신륵사+명성황후생가+영릉+황학산수목원,경기도 여주시 신륵사길 7
1361,김해분청도자기축제,2022-10-21,2022-10-30,김해분청도자박물관+클레이아크미술관,경상남도 김해시 진례면 진례로 275-35
1362,청송사과축제,2022-11-02,2022-11-06,NaN,경상북도 청송군 청송읍 금월로 269


In [37]:
# 축제기간 일수 확인을 위한 컬럼 생성 및 값 추가
festival['Days'] = festival['축제종료일자'] - festival['축제시작일자']


In [38]:
# 데이터프레임 연결을 위한 주소 분리 및 광역시명 컬럼 생성
festival['areaNm'] = festival['소재지도로명주소'].str.split(' ',n=1,expand=True)[0]

In [39]:
# 불필요해진 컬럼 삭제
festival.drop(columns=['관련정보', '소재지도로명주소'], inplace=True)
festival

,축제명,축제시작일자,축제종료일자,Days,areaNm
18,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
21,청송사과축제,2022-11-09,2022-11-13,4 days,NaN
23,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
24,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도
25,산타축제,2022-12-23,2022-12-25,2 days,전라북도
...,...,...,...,...,...
1358,광주세계김치축제,2022-10-20,2022-10-23,3 days,광주광역시
1360,여주오곡나루축제,2022-10-21,2022-10-23,2 days,경기도
1361,김해분청도자기축제,2022-10-21,2022-10-30,9 days,경상남도
1362,청송사과축제,2022-11-02,2022-11-06,4 days,경상북도


In [40]:
# 광역시 종류 확인
festival['areaNm'].unique()

array(['전라남도', nan, '전라북도', '울산광역시', '광주광역시', '경기도', '인천광역시', '강원도',
       '서울특별시', '충청북도', '충청남도', '제주특별자치도', '경상남도', '부산광역시', '경상북도',
       '대전광역시', '세종특별자치시', '세종특별자치시조치원읍문예회관길22', '미정', '울릉군', '대구광역시'],
      dtype=object)

In [41]:
# 오류값 제거
festival = festival.loc[(festival['areaNm'] != 'nan') & 
                        (festival['areaNm'] != '세종특별자치시조치원읍문예회관길22') & 
                        (festival['areaNm'] != '울릉군') & 
                        (festival['areaNm'] != '미정')]

In [42]:
# null 값 제거
festival.dropna(axis=0, inplace=True)

In [43]:
# 축제기간이 한달 밑인 축제만 남기기
festival = festival.loc[festival['Days'] <= '30 Days']
festival

,축제명,축제시작일자,축제종료일자,Days,areaNm
18,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
23,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
24,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도
25,산타축제,2022-12-23,2022-12-25,2 days,전라북도
29,울산쇠부리축제,2022-05-13,2022-05-15,2 days,울산광역시
...,...,...,...,...,...
1358,광주세계김치축제,2022-10-20,2022-10-23,3 days,광주광역시
1360,여주오곡나루축제,2022-10-21,2022-10-23,2 days,경기도
1361,김해분청도자기축제,2022-10-21,2022-10-30,9 days,경상남도
1362,청송사과축제,2022-11-02,2022-11-06,4 days,경상북도


In [44]:
# 인덱스 번호 맞추기
festival.set_index([pd.Index(range(0, 317))], inplace=True)

In [45]:
# 컬럼명 변경
festival.rename(columns={'축제명':'festivalNm', '축제시작일자':'fStart', '축제종료일자':'fClose'}, inplace=True)

In [46]:
festival.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   festivalNm  317 non-null    object         
 1   fStart      317 non-null    datetime64[ns] 
 2   fClose      317 non-null    datetime64[ns] 
 3   Days        317 non-null    timedelta64[ns]
 4   areaNm      317 non-null    object         
dtypes: datetime64[ns](2), object(2), timedelta64[ns](1)
memory usage: 12.5+ KB


In [47]:
# 확인
festival

,festivalNm,fStart,fClose,Days,areaNm
0,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
1,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
2,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도
3,산타축제,2022-12-23,2022-12-25,2 days,전라북도
4,울산쇠부리축제,2022-05-13,2022-05-15,2 days,울산광역시
...,...,...,...,...,...
312,광주세계김치축제,2022-10-20,2022-10-23,3 days,광주광역시
313,여주오곡나루축제,2022-10-21,2022-10-23,2 days,경기도
314,김해분청도자기축제,2022-10-21,2022-10-30,9 days,경상남도
315,청송사과축제,2022-11-02,2022-11-06,4 days,경상북도


In [48]:
# csv 저장
festival.to_csv("./data/festival.csv", index=False, encoding='utf-8')

22년도 공휴일 데이터

In [49]:
with open('c:/Kcoding/MiniProject/APIkey2.txt') as file:
    APIkey2 = file.read()

url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
params ={'serviceKey' : APIkey2, 'solYear' : '2022', 'numOfRows' : '50'}
row_list = []
value_list = []

response = requests.get(url, params=params)
time.sleep(2)
contents = response.text
xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
rows = xml_obj.findAll('item')
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    for j in range(0,len(columns)):
        value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list=[]
holiday = pd.DataFrame(row_list)

100%|██████████| 19/19 [00:00<?, ?it/s]


In [50]:
holiday

,0,1,2,3,4
0,01,1월1일,Y,20220101,1
1,01,설날,Y,20220131,1
2,01,설날,Y,20220201,1
3,01,설날,Y,20220202,1
4,01,삼일절,Y,20220301,1
5,01,대통령선거일,Y,20220309,1
6,01,어린이날,Y,20220505,1
7,01,부처님오신날,Y,20220508,1
8,01,전국동시지방선거,Y,20220601,1
9,01,현충일,Y,20220606,1


In [51]:
holiday.drop(columns=[0, 2, 4], inplace=True)
holiday.rename(columns={1 : 'holNm', 3 : 'baseYmd'}, inplace=True)
holiday['baseYmd'] = pd.to_datetime(holiday['baseYmd'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
holiday['baseYmd'] = pd.to_datetime(holiday['baseYmd'])
holiday

,holNm,baseYmd
0,1월1일,2022-01-01
1,설날,2022-01-31
2,설날,2022-02-01
3,설날,2022-02-02
4,삼일절,2022-03-01
5,대통령선거일,2022-03-09
6,어린이날,2022-05-05
7,부처님오신날,2022-05-08
8,전국동시지방선거,2022-06-01
9,현충일,2022-06-06


In [52]:
holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   holNm    19 non-null     object        
 1   baseYmd  19 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 432.0+ bytes


In [53]:
holiday.to_csv('./data/holiday.csv', encoding='utf-8', index=False)

코로나 확진자 데이터

In [63]:
import datetime

start_date = datetime.date(2022, 1, 1)
end_date = datetime.date(2022, 12, 31)
delta = datetime.timedelta(days=1)

date_list = []
while start_date <= end_date:
    date_list.append(start_date.strftime("%Y-%m-%d"))
    start_date += delta
    
print(date_list)


['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26', '2022-02-27', '2022-02-28', '2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-05', '2022-03-06', '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11', '2022-03-12', '2022

In [64]:
gubun_list = ['전남', '인천', '울산', '경남', '대구', '부산', '세종', '서울', '충북', '전북', '경기', '강원', '대전', '광주', '제주', '경북', '충남']

In [65]:
with open('d:/workspace/MiniProject/APIkey2.txt') as file:
    APIkey2 = file.read()

In [ ]:
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'
row_list = []
value_list = []


for gubun in tqdm(gubun_list):
    for date in date_list:
        params = {'serviceKey' : APIkey2, 'pageNo' : '1', 'numOfRows' : '30', 'apiType' : 'xml', 'std_day' : date, 'gubun' : gubun }
        response = requests.get(url, params=params)
        time.sleep(5)
        contents = response.text
        xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
        rows = xml_obj.findAll('item')
        for i in range(0, len(rows)):
            columns = rows[i].find_all()
            for j in range(0,len(columns)):
                value_list.append(columns[j].text)
            row_list.append(value_list)
            value_list=[]

In [26]:
covid19 = pd.DataFrame(row_list)

In [27]:
covid19

,0,1,2,3,4,5,6,7,8,9,10,11
0,18,2256,전남,全南,Jeollanam-do,18,2021,217,18,0,121.84,2021-08-13
1,0,9,전남,全南,Jeollanam-do,0,3,6,0,0,0.48,2020-03-31
2,17,2022,전남,全南,Jeollanam-do,19,1769,236,17,2,109.21,2021-07-30
3,30,5480,전남,全南,Jeollanam-do,43,4608,0,43,0,296,2021-12-19
4,730,838295,전남,全南,Jeollanam-do,1028,0,0,1028,0,45738,2022-10-25
...,...,...,...,...,...,...,...,...,...,...,...,...
990,8,748,충남,忠南,Chungcheongnam-do,15,590,150,15,0,35.24,2020-11-20
991,0,146,충남,忠南,Chungcheongnam-do,0,142,4,0,0,6.88,2020-06-01
992,1105,931528,충남,忠南,Chungcheongnam-do,5006,0,0,4971,35,43955,2022-08-31
993,0,136,충남,忠南,Chungcheongnam-do,1,99,37,1,0,6.41,2020-04-06


In [28]:
covid19 = covid19.loc[covid19[11].str.contains('2022')]

In [29]:
# 중복값 제거 및 인덱스 재설정
covid19.drop_duplicates(ignore_index=True, inplace=True)

In [30]:
covid19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       320 non-null    object
 1   1       320 non-null    object
 2   2       320 non-null    object
 3   3       320 non-null    object
 4   4       320 non-null    object
 5   5       320 non-null    object
 6   6       320 non-null    object
 7   7       320 non-null    object
 8   8       320 non-null    object
 9   9       320 non-null    object
 10  10      320 non-null    object
 11  11      320 non-null    object
dtypes: object(12)
memory usage: 30.1+ KB


In [31]:
covid19[2].unique()

array(['전남', '인천', '울산', '경남', '대구', '부산', '세종', '충북', '전북', '경기', '강원',
       '대전', '광주', '제주', '경북', '충남'], dtype=object)

In [32]:
# 광역시명 변경 및 합계, 검역 값 행 제거
covid19 = covid19.loc[(covid19[2] != '검역') & (covid19[2] != '합계')]
covid19.replace('경기', '경기도', inplace=True)
covid19.replace('강원', '강원도', inplace=True)
covid19.replace('경북', '경상북도', inplace=True)
covid19.replace('대전', '대전광역시', inplace=True)
covid19.replace('울산', '울산광역시', inplace=True)
covid19.replace('대구', '대구광역시', inplace=True)
covid19.replace('제주', '제주특별자치도', inplace=True)
covid19.replace('충북', '충청북도', inplace=True)
covid19.replace('세종', '세종특별자치시', inplace=True)
covid19.replace('광주', '광주광역시', inplace=True)
covid19.replace('서울', '서울특별시', inplace=True)
covid19.replace('전북', '전라북도', inplace=True)
covid19.replace('부산', '부산광역시', inplace=True)
covid19.replace('경남', '경상남도', inplace=True)
covid19.replace('충남', '충청남도', inplace=True)
covid19.replace('인천', '인천광역시', inplace=True)
covid19.replace('전남', '전라남도', inplace=True)

In [33]:
# 불필요한 컬럼 삭제 및 컬럼명 변경
covid19.drop(columns=[0, 3, 4, 7, 10], inplace=True)
covid19.rename(columns={1:'AccumulateCovid19', 2:'areaNm', 5:'increase', 6:'AccumulateReleased', 8:'localIncrease', 9:'CovidForeigner', 11:'baseYmd'}, inplace=True)
covid19

,AccumulateCovid19,areaNm,increase,AccumulateReleased,localIncrease,CovidForeigner,baseYmd
0,838295,전라남도,1028,0,1028,0,2022-10-25
1,602983,전라남도,148,0,147,1,2022-07-04
2,111483,전라남도,7957,6956,7953,4,2022-03-09
3,6126,전라남도,48,5404,47,1,2022-01-01
4,128930,전라남도,8987,6956,8984,3,2022-03-11
...,...,...,...,...,...,...,...
315,1076452,충청남도,3072,0,3072,0,2022-11-23
316,1011746,충청남도,421,0,418,3,2022-10-17
317,715549,충청남도,193,0,193,0,2022-06-06
318,1118659,충청남도,2252,0,2250,2,2022-12-11


In [34]:
covid19['baseYmd'] = pd.to_datetime(covid19['baseYmd'])

In [24]:
covid19.to_csv('./data/covid19(2).csv', encoding='utf-8', index=False)

미세먼지 데이터

In [66]:
with open('c:/Kcoding/MiniProject/APIkey2.txt') as file:
    APIkey2 = file.read()

row_list = []
value_list = []
url = 'http://apis.data.go.kr/B552584/UlfptcaAlarmInqireSvc/getUlfptcaAlarmInfo'
params = {'serviceKey' : APIkey2, 'returnType' : 'xml', 'numOfRows' : '12000', 'pageNo' : '1', 'year' : '2022'}

response = requests.get(url, params=params)
time.sleep(2)
contents = response.text
xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
rows = xml_obj.findAll('item')
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    for j in range(0,len(columns)):
        value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list=[]

In [67]:
pm = pd.DataFrame(row_list)

In [68]:
pm

,0,1,2,3,4,5,6,7,8,9,10,11
0,29,373,충북,2022-12-27,75,14:00,2022-12-28,2022-12-27,북부권역,13:00,주의보,PM25
1,64,370,제주,2022-12-14,213,23:00,2022-12-14,2022-12-13,제주권역,01:00,주의보,PM10
2,136,308,경기,2022-12-13,311,06:00,2022-12-13,2022-12-13,중부권,16:00,경보,PM10
3,99,364,강원,2022-12-13,145,18:00,2022-12-13,2022-12-13,영서북부,19:00,주의보,PM10
4,76,357,인천,2022-12-13,105,17:00,2022-12-13,2022-12-13,서부권역,18:00,주의보,PM10
...,...,...,...,...,...,...,...,...,...,...,...,...
278,33,5,인천,2022-01-08,86,23:00,2022-01-11,2022-01-08,강화권역,02:00,주의보,PM25
279,27,11,인천,2022-01-08,83,00:00,2022-01-10,2022-01-09,동남부권역,20:00,주의보,PM25
280,23,1,서울,2022-01-08,82,22:00,2022-01-10,2022-01-08,서울권역,21:00,주의보,PM25
281,30,2,경기,2022-01-08,81,22:00,2022-01-11,2022-01-08,북부권,03:00,주의보,PM25


In [69]:
# 불필요한 컬럼 삭제 및 컬럼명 변경
pm.drop(columns=[0, 1, 5, 6, 7, 8, 9],inplace=True)
pm.rename(columns={2 : 'areaNm', 3 : 'baseYmd', 4 : 'CAI', 10 : 'advisory', 11 : 'pmcategory'}, inplace=True)

In [70]:
pm['areaNm'].unique()

array(['충북', '제주', '경기', '강원', '인천', '전남', '경북', '경남', '서울', '광주', '대전',
       '전북', '충남', '부산', '대구', '울산', '세종'], dtype=object)

In [71]:
pm.replace('경기', '경기도', inplace=True)
pm.replace('강원', '강원도', inplace=True)
pm.replace('경북', '경상북도', inplace=True)
pm.replace('대전', '대전광역시', inplace=True)
pm.replace('울산', '울산광역시', inplace=True)
pm.replace('대구', '대구광역시', inplace=True)
pm.replace('제주', '제주특별자치도', inplace=True)
pm.replace('충북', '충청북도', inplace=True)
pm.replace('세종', '세종특별자치시', inplace=True)
pm.replace('광주', '광주광역시', inplace=True)
pm.replace('서울', '서울특별시', inplace=True)
pm.replace('전북', '전라북도', inplace=True)
pm.replace('부산', '부산광역시', inplace=True)
pm.replace('경남', '경상남도', inplace=True)
pm.replace('충남', '충청남도', inplace=True)
pm.replace('인천', '인천광역시', inplace=True)
pm.replace('전남', '전라남도', inplace=True)

In [72]:
# 년월일 타입 변경
pm['baseYmd'] = pd.to_datetime(pm['baseYmd'])

In [73]:
# 중복값 확인
pm.duplicated().sum()

18

In [74]:
# 중복값 제거
pm.drop_duplicates(ignore_index=True, inplace=True)

In [75]:
# 확인
pm

,areaNm,baseYmd,CAI,advisory,pmcategory
0,충청북도,2022-12-27,75,주의보,PM25
1,제주특별자치도,2022-12-14,213,주의보,PM10
2,경기도,2022-12-13,311,경보,PM10
3,강원도,2022-12-13,145,주의보,PM10
4,인천광역시,2022-12-13,105,주의보,PM10
...,...,...,...,...,...
260,인천광역시,2022-01-08,86,주의보,PM25
261,인천광역시,2022-01-08,83,주의보,PM25
262,서울특별시,2022-01-08,82,주의보,PM25
263,경기도,2022-01-08,81,주의보,PM25


In [76]:
pm.to_csv('./data/pm.csv', index=False, encoding='utf-8')

데이터 합치기

In [77]:
# 관광객수 집계 데이터
df.head(3)

,areaNm,daywkDivNm,baseYmd,local,outside,foreigner,total_tourist
0,서울특별시,토요일,2022-01-01,3698523.5,1171117.5,3806.877858,4.873448e+06
1,부산광역시,토요일,2022-01-01,1206936.5,445102.0,2165.794258,1.654204e+06
2,대구광역시,토요일,2022-01-01,1013701.5,264113.5,218.422286,1.278033e+06


In [78]:
# 날씨 데이터
weather.head(3)

,areaNm,baseYmd,avgTemp,dayRf,daySf
0,강원도,2022-01-01,-8.2,0.0,0.0
1,강원도,2022-01-02,-2.7,0.0,0.0
2,강원도,2022-01-03,-3.8,0.0,0.0


In [79]:
# 지역축제 데이터
festival.head(3)

,festivalNm,fStart,fClose,Days,areaNm
0,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
1,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
2,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도


In [80]:
# 코로나 데이터
covid19.head(3)

,AccumulateCovid19,areaNm,increase,AccumulateReleased,localIncrease,CovidForeigner,baseYmd
0,6923,울산광역시,50,6371,50,0,2022-01-01
1,2158,세종특별자치시,16,1957,16,0,2022-01-01
2,11602,충청북도,104,10540,103,1,2022-01-01


In [81]:
# 미세먼지 데이터
pm.head(3)

,areaNm,baseYmd,CAI,advisory,pmcategory
0,충청북도,2022-12-27,75,주의보,PM25
1,제주특별자치도,2022-12-14,213,주의보,PM10
2,경기도,2022-12-13,311,경보,PM10


In [82]:
# 관광객 집계 데이터와 날씨 데이터 합치기
result = pd.merge(df, weather, left_on=['baseYmd', 'areaNm'], right_on=['baseYmd', 'areaNm'], how='left')
result

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

In [ ]:
# 축제여부 컬럼을 만들고 지역축제 데이터를 이용하여 축제여부 데이터 입력
for index, row in tqdm(result.iterrows()):
    for index2, row2 in festival.iterrows():
        if row['areaNm'] == row2['areaNm']:
            if (row2['fStart'] <= row['baseYmd']) and (row['baseYmd'] <= row2['fClose']):
                result.at[index, 'festival'] = 1
            else:
                pass
        else:
            pass
# 결측값 0으로 채우기
result.fillna(0, inplace=True)
result = result.astype({'festival':'int'})

6086it [01:05, 92.86it/s] 


In [ ]:
# 공휴일 추가
result['holiday'] = 0

for index, row in tqdm(result.iterrows()):
    if row['daywkDivNm'] == '토요일' or row['daywkDivNm'] == '일요일':
        result.at[index, 'holiday'] = 1
    for index2, row2 in holiday.iterrows():
        if row['baseYmd'] == row2['baseYmd']:
            result.at[index, 'holiday'] = 1
        else:
            pass

6086it [00:07, 806.99it/s]


In [ ]:
# 적용 확인
# 2022-08-15 / 월요일이지만 광복절이므로 휴무일
result.loc[result['baseYmd'] == '2022-08-15']

,areaNm,daywkDivNm,baseYmd,local,outside,foreigner,total_tourist,avgTemp,dayRf,daySf,festival,holiday
3842,서울특별시,월요일,2022-08-15,4202604.0,1320945.5,4734.649213,5.528284e+06,28.1,0.9,0.0,0,1
3843,부산광역시,월요일,2022-08-15,1434800.5,477795.5,703.699933,1.913300e+06,29.0,0.0,0.0,0,1
3844,대구광역시,월요일,2022-08-15,1077121.5,297668.0,188.448266,1.374978e+06,30.9,0.0,0.0,0,1
3845,인천광역시,월요일,2022-08-15,1189559.0,479258.5,4252.724591,1.673070e+06,27.2,0.0,0.0,0,1
3846,광주광역시,월요일,2022-08-15,596792.0,175934.5,87.717665,7.728142e+05,29.4,0.0,0.0,0,1
3847,대전광역시,월요일,2022-08-15,672994.0,213062.5,182.870701,8.862394e+05,28.1,7.0,0.0,0,1
3848,울산광역시,월요일,2022-08-15,515553.5,158473.0,197.028239,6.742235e+05,30.2,0.0,0.0,1,1
3849,세종특별자치시,월요일,2022-08-15,161476.0,77794.5,84.225455,2.393547e+05,28.2,29.8,0.0,0,1
3850,경기도,월요일,2022-08-15,5985962.5,1835466.5,2349.948910,7.823779e+06,27.7,2.9,0.0,0,1
3851,강원도,월요일,2022-08-15,850640.0,742500.5,421.215460,1.593562e+06,27.5,35.1,0.0,1,1


In [ ]:
# 코로나 확진자 데이터 추가
result = pd.merge(result, covid19, left_on=['baseYmd', 'areaNm'], right_on=['baseYmd', 'areaNm'], how='left')

In [ ]:
# 미세먼지 데이터 추가
result['PM10'] = 0
result['PM25'] = 0

for row in tqdm(result.itertuples(), total=result.shape[0]):
    for row2 in pm.itertuples():
        if row2.pmcategory == 'PM10':
            if row.areaNm == row2.areaNm and row.baseYmd == row2.baseYmd:
                result.loc[row.Index, 'PM10'] = 1
        elif row2.pmcategory == 'PM25':
            if row.areaNm == row2.areaNm and row.baseYmd == row2.baseYmd:
                result.loc[row.Index, 'PM25'] = 1


100%|██████████| 6218/6218 [00:03<00:00, 1647.01it/s]


In [ ]:
# 미세먼지 데이터 추가 확인
result.loc[(result['PM25'] == 1) | (result['PM10'] == 1)]

,areaNm,daywkDivNm,baseYmd,local,outside,foreigner,total_tourist,avgTemp,dayRf,daySf,festival,holiday,AccumulateCovid19,increase,AccumulateReleased,localIncrease,CovidForeigner,PM10,PM25
119,서울특별시,토요일,2022-01-08,4141599.0,1551105.5,3688.801782,5.696393e+06,0.3,0.0,0.0,0,1,234449,994,188072,933,61,0,1
122,인천광역시,토요일,2022-01-08,1159525.5,434420.0,2817.468483,1.596763e+06,1.6,0.0,0.0,0,1,38491,203,33916,193,10,0,1
127,경기도,토요일,2022-01-08,5709655.0,1711720.5,3793.777429,7.425169e+06,-2.4,0.0,0.0,0,1,196357,1208,165430,1168,40,0,1
141,대전광역시,일요일,2022-01-09,625859.5,196164.5,480.200925,8.225042e+05,2.5,0.0,0.0,0,1,12665,40,11243,35,5,0,1
143,세종특별자치시,일요일,2022-01-09,132587.0,62573.0,39.354514,1.951994e+05,1.8,0.0,0.0,0,1,2265,13,2113,13,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5908,전라남도,화요일,2022-12-13,1035219.5,233338.0,230.821333,1.268788e+06,6.9,0.0,0.0,0,0,906670,2762,0,2762,0,1,0
5909,경상북도,화요일,2022-12-13,1499547.0,333417.0,261.350765,1.833225e+06,1.8,4.2,0.0,0,0,NaN,NaN,NaN,NaN,NaN,1,0
5910,경상남도,화요일,2022-12-13,1749461.0,291889.5,457.266431,2.041808e+06,4.5,0.2,0.0,0,0,1653612,5304,0,5302,2,1,0
5928,제주특별자치도,수요일,2022-12-14,567679.0,145915.0,212.590887,7.138066e+05,4.2,2.1,0.0,1,0,NaN,NaN,NaN,NaN,NaN,1,0


In [ ]:
# 중복 확인
result.duplicated().sum()

2

In [ ]:
# 중복 제거
result.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
# 결측값 확인
result.isnull().sum()

areaNm                   0
daywkDivNm               0
baseYmd                  0
local                    0
outside                  0
foreigner                0
total_tourist            0
avgTemp                  0
dayRf                    0
daySf                    0
festival                 0
holiday                  0
AccumulateCovid19     1383
increase              1383
AccumulateReleased    1383
localIncrease         1383
CovidForeigner        1383
PM10                     0
PM25                     0
dtype: int64

In [ ]:
result.info()

In [ ]:
# 확인
result.head(3)

In [ ]:
# 소숫점 삭제 및 정수형 변환
result['Local_count'] = round(result['local'])
result['Outside_count'] = round(result['outside'])
result['Foreigner_count'] = round(result['foreigner'])
result['Total_count'] = round(result['total_tourist'])
result.drop(columns=['local', 'outside', 'foreigner', 'total_tourist'], inplace=True)
result = result.astype({'Local_count':'int', 'Outside_count':'int', 'Foreigner_count':'int', 'Total_count':'int'})

In [ ]:
# 최종 확인
result

In [ ]:
# 전처리 최종 결과 csv 저장
result.to_csv('./data/result.csv', encoding='utf-8', index=False)